# For your ease - copy and paste in the demo session

You can develop your code in Google Colab initially, and then copy and paste it into the `.py` file for use with Streamlit.

This Google Colab Notebook shows you how the code looks like in Google Colab, and demonstrates how easily you could copy the code into your `.py` files.

Entire code of the streamlit demo app: https://github.com/hkust-dh/streamlit-demo-20240322/

# File [`2_Import Data via API.py`](https://github.com/hkust-dh/streamlit-demo-20240322/blob/main/pages/2_Import%20Data%20via%20API.py)

In [ ]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

In [ ]:
# Send a GET request to the URL and retrieve the XML content
#https://lbezone.hkust.edu.hk/rse/western-collection

url = "https://lbezone.hkust.edu.hk/rse/OAI/Server.php?verb=ListRecords&metadataPrefix=oai_dc&set=western_collection"
response = requests.get(url)
response.encoding = 'utf-8'
xml_content = response.content

# Parse the XML content
root = ET.fromstring(xml_content)

# Create lists to store the extracted information
titles = []
dates = []
languages = []
urls = []
thumb_urls =[]

# Iterate over each record and extract the desired information
for record in root.findall(".//{http://www.openarchives.org/OAI/2.0/}record"):
    title = record.find(".//{http://purl.org/dc/elements/1.1/}title").text
    date = record.find(".//{http://purl.org/dc/elements/1.1/}date").text
    lang = record.find(".//{http://purl.org/dc/elements/1.1/}language").text

    titles.append(title)
    dates.append(date)
    languages.append(lang)

    identifiers = record.findall('.//{http://purl.org/dc/elements/1.1/}identifier')
    thumb_url = identifiers[1].text
    thumb_urls.append(thumb_url)

    doi_url = identifiers[4].text
    urls.append(doi_url)

# Create a dataframe from the extracted information
data = {
    "Title": titles,
    "Year": dates,
    "Language": languages,
    "URL": urls,
    "Thumbnail image": thumb_urls
}
data = pd.DataFrame(data)
# in streamlit ----> st.dataframe(data, hide_index=True)

In [ ]:
data.head()

,Title,Year,Language,URL,Thumbnail image
0,An authentic account of an embassy from the Ki...,1799,English,https://doi.org/10.14711/spcol/b384524,https://lbezone.hkust.edu.hk/thumb/thumb-b3845...
1,Eighteen capitals of China,1911,English,https://doi.org/10.14711/spcol/b722488,https://lbezone.hkust.edu.hk/thumb/thumb-b7224...
2,A dissertation on the nature and character of ...,1838,English,https://doi.org/10.14711/spcol/b477387,https://lbezone.hkust.edu.hk/thumb/thumb-b4773...
3,Het gezantschap der Neêrlandtsche Oost-Indisch...,1665,Dutch,https://doi.org/10.14711/spcol/b498064,https://lbezone.hkust.edu.hk/thumb/thumb-b4980...
4,John Huighen van Linschoten : his discours of ...,1598,English,https://doi.org/10.14711/spcol/b456064,https://lbezone.hkust.edu.hk/thumb/thumb-b4560...


In [ ]:
data.to_csv('demo-data.csv', index=False, encoding='utf-8')

In [ ]:
data.to_excel('demo-data.xlsx', index=False)

# File [`3_Analysis and Display.py`](https://github.com/hkust-dh/streamlit-demo-20240322/blob/main/pages/3_Analysis%20and%20Display.py)

In [ ]:
len(data)

139

In [ ]:
booknum = len(data)
booknum

139

In [ ]:
print(f'There are a total of {booknum} books in this collection online as of 19 March 2024.')

There are a total of 139 books in this collection online as of 19 March 2024.


in streamlit
```
booknum = len(data)
st.metric(label="No. of books", value=booknum)
st.write(f'There are a total of {booknum} books in this collection online as of 19 March 2024.')
```

## Charts

In [ ]:
# Count the occurrences of each language
language_counts = data['Language'].value_counts()
language_counts

English    127
French       6
German       4
Dutch        1
Chinese      1
Name: Language, dtype: int64

In [ ]:
df_languages = pd.DataFrame({'Language': language_counts.index, 'Count': language_counts.values})
df_languages.head()

,Language,Count
0,English,127
1,French,6
2,German,4
3,Dutch,1
4,Chinese,1


In [ ]:
import plotly.express as px

# Create a pie chart using Plotly Express
fig = px.pie(df_languages, values='Count', names='Language', title='Distribution of Languages')
fig

In streamlit
```
# Count the occurrences of each language
language_counts = data['Language'].value_counts()

# Create a DataFrame from the counts
df_languages = pd.DataFrame({'Language': language_counts.index, 'Count': language_counts.values})

# Create a pie chart using Plotly Express
fig = px.pie(df_languages, values='Count', names='Language', title='Distribution of Languages')

# Display the pie chart in Streamlit
st.plotly_chart(fig)
```

## Using `for loop` to display thumbnails like catalog/gallery

In [ ]:
for index, row in data.iterrows():
    title = row["Title"]
    year = row["Year"]
    lang = row["Language"]
    url = row["URL"]
    image = row["Thumbnail image"]

    print(title)
    print(year)
    print(lang)
    print(url)
    print(image)
    print("========")

An authentic account of an embassy from the King of Great Britain to the Emperor of China
1799
English
https://doi.org/10.14711/spcol/b384524
https://lbezone.hkust.edu.hk/thumb/thumb-b384524.jpg
Eighteen capitals of China
1911
English
https://doi.org/10.14711/spcol/b722488
https://lbezone.hkust.edu.hk/thumb/thumb-b722488.jpg
A dissertation on the nature and character of the Chinese system of writing : in a letter to John Vaughan
1838
English
https://doi.org/10.14711/spcol/b477387
https://lbezone.hkust.edu.hk/thumb/thumb-b477387.jpg
Het gezantschap der Neêrlandtsche Oost-Indische Compagnie, aan den grooten Tartarischen Cham, den tegenwoordigen keizer van China : waar in de gedenkwaerdighste geschiedenissen, die onder het reizen door de Sineesche landtschappen, Quantung, Kiangsi, Nanking, Xantung en Peking, en aan het keizerlijke hof te Peking, sedert den jare 1655 tot 1657 zijn voorgevallen, op het bondigste verhandelt worden ...
1665
Dutch
https://doi.org/10.14711/spcol/b498064
https:/

in streamlit
```
colnum = 6
for i in range(0, len(data), colnum):
    row = data.iloc[i:i+colnum]

    columns = st.columns(colnum)
    for index, row in row.iterrows():
        title = row["Title"]
        year = row["Year"]
        lang = row["Language"]
        url = row["URL"]
        image = row["Thumbnail image"]

        with columns[index % colnum]:
            st.image(image, caption=f"Year: {year} | {lang}")
            with st.expander("Title"):
                st.write(title)
            st.write(url)
            st.markdown("---")
```

# File [`7__Guess a number (Traditional Chatbot).py`](https://github.com/hkust-dh/streamlit-demo-20240322/blob/main/pages/7__Guess%20a%20number%20(Traditional%20Chatbot).py)

In [ ]:
number_to_guess = 3

while True:
  try:
      user_input = int(input("Let's guess! Input a number here: "))

      if user_input == number_to_guess:
          print("Superb! You guessed the number correctly.")
          break
      elif user_input < number_to_guess:
          print("Too low! Try again.")
      elif user_input > number_to_guess:
          print("Too high! Try again.")
      else:
          print("Please enter a number.")

  except ValueError:
      print("Invalid input. Please enter a number (integer).")

Let's guess! Input a number here: 5
Too high! Try again.
Let's guess! Input a number here: 2
Too low! Try again.
Let's guess! Input a number here: hello
Invalid input. Please enter a number (integer).
Let's guess! Input a number here: 4
Too high! Try again.
Let's guess! Input a number here: 4.3
Invalid input. Please enter a number (integer).
Let's guess! Input a number here: 3
Superb! You guessed the number correctly.


In [ ]:
import random

number_to_guess = random.randint(1, 10)

while True:
  try:
      user_input = int(input("Let's guess! Input a number here: "))

      if user_input == number_to_guess:
          print("Superb! You guessed the number correctly.")
          number_to_guess = random.randint(1, 10)
      elif user_input < number_to_guess:
          print("Too low! Try again.")
      elif user_input > number_to_guess:
          print("Too high! Try again.")
      else:
          print("Please enter a number.")

  except ValueError:
      print("Invalid input. Please enter a number (integer).")

Let's guess! Input a number here: 9
Too high! Try again.
Let's guess! Input a number here: 7
Too high! Try again.
Let's guess! Input a number here: 6
Too high! Try again.
Let's guess! Input a number here: 3
Too high! Try again.
Let's guess! Input a number here: 1
Superb! You guessed the number correctly.
Let's guess! Input a number here: 1
Too low! Try again.
Let's guess! Input a number here: 4
Too high! Try again.
Let's guess! Input a number here: 5
Too high! Try again.
Let's guess! Input a number here: 3
Too high! Try again.
Let's guess! Input a number here: 2
Superb! You guessed the number correctly.


KeyboardInterrupt: Interrupted by user

in streamlit
```
# Generate a random number between 1 and 10 as the answer
if 'number_to_guess' not in st.session_state:
    st.session_state.number_to_guess = random.randint(1, 10)

with st.expander("Click here to see the answer"):
    st.write(st.session_state.number_to_guess)

# Start or retrieve session
session_state = st.session_state.get("session_state", {"conversation": []})

# Retrieve conversation from session state
conversation = session_state["conversation"]

messages = st.container(height=500)

# Check if there is no existing conversation
if not conversation:
    # Add default assistant message
    conversation.append(("assistant", "Let's start guessing a number!"))
    for role, content in conversation:
        messages.chat_message(role).write(content)

# React to user input
if prompt := st.chat_input("Let's guess! Input a number here."):
    # Check if the input is a number
    try:
        user_input = int(prompt)
        conversation.append(("user", prompt))

        # Check if the guess is correct
        if user_input == st.session_state.number_to_guess:
            response = "Superb! You guessed the number correctly. A fresh number has been generated. Let's begin another round of guessing right away!"
            # Generate a new random number for the next round
            st.session_state.number_to_guess = random.randint(1, 10)
        elif user_input < st.session_state.number_to_guess:
            response = "Too low! Try again."
        elif user_input > st.session_state.number_to_guess:
            response = "Too high! Try again."
        else:
            response = "Please enter a number."

        # Add assistant response to conversation
        conversation.append(("assistant", response))

    except ValueError:
        response = "Invalid input. Please enter a number (integer)."
        conversation.append(("assistant", response))

    # Update conversation in session state
    session_state["conversation"] = conversation

    # Display conversation history
    for role, content in conversation:
        messages.chat_message(role).write(content)

# Update session state
st.session_state["session_state"] = session_state
```